In [4]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime

In [8]:
#신세계푸드 031440 / 신세계건설 034300 / 신세계 I&C 035510
# 광주신세계 037710 / (주식회사)신세계 004170 / (주식회사)이마트 139480 / 현대건설 000720 / CJ제일제당 097950
codes=['031440','034300','035510','037710','004170','139480','000720','097950']

In [ ]:
forders = os.listdir('./data/')
print(forders)

In [25]:
# '2022_10', '2022_11', '2022_12', '2022_7', '2022_8', '2022_9', '2023_1', '2023_2', '2023_3', '2023_4', '2023_5', '2023_6'
for forder in forders:

    # '031440','034300','035510','037710','004170','139480','000720','097950'
    for code in codes:

        globals()['df_{}_{}'.format(forder,code)] = pd.DataFrame()

        for file_path in glob.glob('./data/{}/{}_*.csv'.format(forder,code)):
           
            df= pd.read_csv(file_path, encoding='utf-8') 
            globals()['df_{}_{}'.format(forder,code)] = pd.concat([globals()['df_{}_{}'.format(forder,code)], df])

        # print('df_{}_{}'.format(forder,code))


In [26]:
# '031440','034300','035510','037710','004170','139480','000720','097950'
for code in codes:

    globals()['df_{}'.format(code)] = pd.DataFrame()
    
    # '2022_10', '2022_11', '2022_12', '2022_7', '2022_8', '2022_9', '2023_1', '2023_2', '2023_3', '2023_4', '2023_5', '2023_6'
    for forder in forders:

        df= globals()['df_{}_{}'.format(forder,code)]
        globals()['df_{}'.format(code)] = pd.concat([globals()['df_{}'.format(code)], df])


In [27]:
for code in codes:

    globals()['df_{}'.format(code)].insert(0,'Datetime', globals()['df_{}'.format(code)]['date'].astype('str') + globals()['df_{}'.format(code)]['time'].astype('str'))

    globals()['df_{}'.format(code)]['Datetime'] = globals()['df_{}'.format(code)]['Datetime'].apply(lambda _ : datetime.strptime(_,'%Y%m%d%H%M%S'))

    globals()['df_{}'.format(code)].sort_values('Datetime',inplace=True,ignore_index=True)

    globals()['df_{}'.format(code)] = globals()['df_{}'.format(code)][['Datetime', 'open', 'high', 'low', 'close', 'qty','value', 'sign']]

    globals()['df_{}'.format(code)].to_csv('./data/df_{}.csv'.format(code))

In [6]:
df = pd.read_csv('./data/df_000720.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188280 entries, 0 to 188279
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Datetime    188280 non-null  object 
 1   open        188280 non-null  float64
 2   high        188280 non-null  float64
 3   low         188280 non-null  float64
 4   close       188280 non-null  float64
 5   qty         188280 non-null  int64  
 6   volume      188280 non-null  int64  
 7   last_price  188280 non-null  float64
 8   sign        188280 non-null  int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 12.2+ MB


In [14]:
for code in codes:
    globals()['df_{}'.format(code)] = pd.read_csv('./data/df_{}.csv'.format(code))
    globals()['df_{}'.format(code)]['Datetime'] = globals()['df_{}'.format(code)]['Datetime'].apply(lambda _ : datetime.strptime(_,'%Y-%m-%d %H:%M:%S'))
    globals()['df_{}'.format(code)].sort_values('Datetime',inplace=True,ignore_index=True)


    # 15시20분 이후 데이터 자르기
    i=0
    for time_ in globals()['df_{}'.format(code)]['Datetime'].apply(lambda _ : _.time()):
        if time_ > datetime.strptime('1520','%H%M').time():
            globals()['df_{}'.format(code)].drop(index=[i],axis=1,inplace=True)
            i+=1
        else:
            i+=1
            
    globals()['df_{}'.format(code)].reset_index(inplace=True,drop=True)
    globals()['df_{}'.format(code)].to_csv('./data/df_{}.csv'.format(code),index=False)
    

In [15]:
df_000720

,Datetime,open,high,low,close,qty,volume,last_price,sign
0,2021-06-28 09:01:00,54973.0,55070.0,54681.0,54973.0,49076,2696800000,0.0,0
1,2021-06-28 09:02:00,54973.0,55168.0,54876.0,55070.0,21315,1172830000,54973.0,0
2,2021-06-28 09:03:00,55070.0,55168.0,54778.0,54778.0,25534,1404920000,54973.0,1
3,2021-06-28 09:04:00,54876.0,54876.0,54486.0,54681.0,20114,1099340000,55070.0,-1
4,2021-06-28 09:05:00,54778.0,54876.0,54584.0,54876.0,15079,825440000,54876.0,-1
...,...,...,...,...,...,...,...,...,...
187975,2023-06-26 15:16:00,40900.0,40950.0,40900.0,40900.0,4061,166094900,40950.0,-1
187976,2023-06-26 15:17:00,40900.0,40950.0,40900.0,40950.0,5610,229449000,40900.0,0
187977,2023-06-26 15:18:00,40950.0,40950.0,40850.0,40900.0,12084,494839800,40900.0,1
187978,2023-06-26 15:19:00,40900.0,41000.0,40900.0,40950.0,12249,500984100,40950.0,-1
